## Programming Assignment: Примеры задач анализа текстов

### Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

In [24]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [2]:
def write_answer(filename, result):
    with open(filename, "w") as fout:
        fout.write(str(result))

### Задание

1.Загрузите датасет. Описание датасета можно посмотреть [здесь](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection).

2.Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [4]:
raw_data = pd.read_csv('SMSSpamCollection.txt', sep='\t', header=None, names=['label', 'text'])
raw_data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
raw_data.shape

(5572, 2)

3.Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [6]:
labels = raw_data.loc[:, 'label']
labels = map(lambda label: 0 if label == 'ham' else 1, labels)

In [7]:
texts = raw_data.loc[:, 'text']

4.Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [8]:
cv = CountVectorizer()
X = cv.fit_transform(texts).toarray()
X.shape

(5572, 8713)

5.Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [9]:
scores = cross_val_score(LogisticRegression(random_state=2), X, labels, scoring='f1', cv=10)
mean_score = scores.mean()

In [10]:
mean_score

0.9326402983610631

In [11]:
write_answer('question5.txt', "%.1f" % mean_score)

6.А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [12]:
predict_to = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB", 
              "FreeMsg: Txt: claim your reward of 3 hours talk time", 
              "Have you visited the last lecture on physics?", 
              "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", 
              "Only 99$"]

In [13]:
clf_pipeline = Pipeline(
    [('vectorizer', CountVectorizer()), 
     ('classifier', LogisticRegression(random_state=2))
    ])

clf_pipeline.fit(texts, labels)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...nalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [14]:
predictions = clf_pipeline.predict(predict_to)
predictions

array([1, 1, 0, 0, 0])

In [15]:
with open('question6.txt', 'w') as fout:
    fout.write(' '.join(str(prediction) for prediction in predictions))

7.Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [16]:
# ngram_range(2,2)
cv = CountVectorizer(ngram_range=(2,2))
X = cv.fit_transform(texts)
print X.shape
scores = cross_val_score(LogisticRegression(random_state=2), X, labels, scoring='f1', cv=10, n_jobs=-1)
mean_score1 = scores.mean()
print "ngram_range(2,2), mean score:", mean_score1

(5572, 41793)
ngram_range(2,2), mean score: 0.822422066419


In [17]:
# ngram_range(3,3)
cv = CountVectorizer(ngram_range=(3,3))
X = cv.fit_transform(texts)
print X.shape
scores = cross_val_score(LogisticRegression(random_state=2), X, labels, scoring='f1', cv=10, n_jobs=-1)
mean_score2 = scores.mean()
print "ngram_range(3,3), mean score:", mean_score2

(5572, 54461)
ngram_range(3,3), mean score: 0.725016155547


In [18]:
# ngram_range(1,3)
cv = CountVectorizer(ngram_range=(1,3))
X = cv.fit_transform(texts)
print X.shape
scores = cross_val_score(LogisticRegression(random_state=2), X, labels, scoring='f1', cv=10, n_jobs=-1)
mean_score3 = scores.mean()
print "ngram_range(2,2), mean score:", mean_score3

(5572, 104967)
ngram_range(2,2), mean score: 0.925138255865


In [19]:
with open('question7.txt', 'w') as fout:
    fout.write(' '.join('%.2f' % mean for mean in [mean_score1, mean_score2, mean_score3]))

8.Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [20]:
# ngram_range(2,2)
cv = CountVectorizer(ngram_range=(2,2))
X = cv.fit_transform(texts)
print X.shape
scores = cross_val_score(MultinomialNB(), X, labels, scoring='f1', cv=10, n_jobs=-1)
mean_score1 = scores.mean()
print "ngram_range(2,2), naive bayes, mean score:", mean_score1

(5572, 41793)
ngram_range(2,2), naive bayes, mean score: 0.645501517799


In [21]:
# ngram_range(3,3)
cv = CountVectorizer(ngram_range=(3,3))
X = cv.fit_transform(texts)
print X.shape
scores = cross_val_score(MultinomialNB(), X, labels, scoring='f1', cv=10, n_jobs=-1)
mean_score2 = scores.mean()
print "ngram_range(3,3), naive bayes, mean score:", mean_score2

(5572, 54461)
ngram_range(3,3), naive bayes, mean score: 0.378719485246


In [22]:
# ngram_range(1,3)
cv = CountVectorizer(ngram_range=(1,3))
X = cv.fit_transform(texts)
print X.shape
scores = cross_val_score(MultinomialNB(), X, labels, scoring='f1', cv=10, n_jobs=-1)
mean_score3 = scores.mean()
print "ngram_range(1,3), naive bayes, mean score:", mean_score3

(5572, 104967)
ngram_range(1,3), naive bayes, mean score: 0.888485965606


In [23]:
with open('question8.txt', 'w') as fout:
    fout.write(' '.join('%.2f' % mean for mean in [mean_score1, mean_score2, mean_score3]))

9.Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [26]:
tfidfv = TfidfVectorizer()
X = tfidfv.fit_transform(texts)
scores = cross_val_score(LogisticRegression(random_state=2), X, labels, scoring='f1', cv=10, n_jobs=-1)
mean_score = scores.mean()
print "tfidf vectorizer, mean score: ", mean_score

tfidf vectorizer, mean score:  0.852859955417


In [27]:
with open('question9.txt', 'w') as fout:
    fout.write(str(-1))